Import

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
vote_df = pd.read_csv('../train.csv')

#투표 여부 비율 확인
print(vote_df.voted.value_counts())

0    19918
1    16507
Name: voted, dtype: int64


In [3]:
#머신러닝 알고리즘에 불필요한 속성 제거
def drop_features(df):
    df.drop(['urban', 'gender', 'race', 'engnat', 'tp01','tp02', 'tp03','tp04','tp05','tp06','tp07','tp08','tp09','tp10',
            'wr_01','wr_02','wr_03','wr_04','wr_05','wr_06','wr_07','wr_08','wr_09',
            'wr_10','wr_11','wr_12','wr_13',
            'wf_01', 'wf_02', 'wf_03','hand'], axis = 1, inplace=True) 
    return df

In [4]:
#Q_E 함수
def replace_feature_Q(df):
    def outlier_iqr(df=None, column= None, weight=1.5):
        data = df[column]
        q25, q75 = np.percentile(data.values, 25), np.percentile(data.values, 75)
        iqr = q75 - q25
        iqr_weight = iqr * weight
        lower = q25 - iqr_weight
        upper = q75 + iqr_weight
        outlier_index = data[(data < lower) | (data > upper)].index
        return outlier_index
    col_name = ['QaE', 'QbE', 'QcE', 'QdE','QeE','QfE','QgE','QhE','QiE','QjE','QkE','QlE','QmE','QnE','QoE','QpE','QqE','QrE','QsE','QtE']
    for col in col_name:
        outlier_index = outlier_iqr(df, column = col)
        for i in outlier_index:
            df[col].replace(df.loc[i, col], df[col].mean())
    
    scaler = MinMaxScaler()
    df_QE = df[col_name]
    scaler.fit(df_QE)
    df[col_name] = scaler.transform(df_QE)
    return df

In [5]:
# age_group 함수
def replace_feature_age(df):
    df['age_group'].replace('+70s', '60s',inplace=True)
    #age_group 레이블 인코딩 수행:
    feature = ['age_group']
    le = LabelEncoder()
    le = le.fit(df[feature])
    df[feature] = le.transform(df[feature])
    return df

In [6]:
#familysize 함수
def fs_incodidng(df):
    def fs_labeling(df):
        label_list=[]
        for i in range(len(df)):
            if df.iloc[i]['familysize']==[2]:
                 label_list.append(7)
            elif df.iloc[i]['familysize']==[3]:
                 label_list.append(2)
            elif df.iloc[i]['familysize']==[1]:
                 label_list.append(5)
            elif df.iloc[i]['familysize']==[4]:
                 label_list.append(4)
            elif df.iloc[i]['familysize']==[5]:
                 label_list.append(3)
            elif df.iloc[i]['familysize']==[0]:
                 label_list.append(2)
            elif df.iloc[i]['familysize']==[6]:
                 label_list.append(1)
            else:
                label_list.append(0)
        return label_list
    vote_df['familysize'] = fs_labeling(vote_df)
    # 라벨링 후 0인 컬럼을 7로 변환(7이 가장 많은 비율을 차지하는 컬럼)
    vote_df['familysize'].replace(0,7,inplace=True)
    return df

In [7]:
# 0값 대체함수
def replace_feature(df):
    df['education'].replace(0,2,inplace=True)
    df['married'].replace(0,1,inplace=True)
    df['familysize'].replace(0,7,inplace=True) 
    return df

In [8]:
# 원핫인코딩 함수
def OH_incording(df):
    df = pd.get_dummies(df, columns = ['religion'])
    return df

In [9]:
def transform_features(df):
    df = replace_feature_Q(df)
    df = replace_feature_age(df)
    df = drop_features(df)
    df = replace_feature(df)
    df = fs_incodidng(df)
    df = OH_incording(df)
    return df

In [10]:
# 원본데이터를 재로딩하고, 함수적용
vote_df = pd.read_csv('../train.csv')
y_vote_df = vote_df['voted']
X_vote_df = vote_df.drop('voted',axis=1)

X_vote_df = transform_features(X_vote_df)

In [11]:
X_vote_df.head(5)

,index,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,...,religion_Buddhist,religion_Christian_Catholic,religion_Christian_Mormon,religion_Christian_Other,religion_Christian_Protestant,religion_Hindu,religion_Jewish,religion_Muslim,religion_Other,religion_Sikh
0,43375,1.0,0.000124,5.0,0.000322,5.0,0.000383,1.0,0.000440,1.0,...,0,1,0,0,0,0,0,0,0,0
1,4486,1.0,0.000211,5.0,0.000473,5.0,0.002294,1.0,0.000690,1.0,...,0,0,0,0,0,1,0,0,0,0
2,10262,5.0,0.000242,1.0,0.000224,2.0,0.000687,1.0,0.001164,3.0,...,0,0,0,1,0,0,0,0,0,0
3,14088,4.0,0.000065,1.0,0.000911,1.0,0.002214,4.0,0.002965,1.0,...,0,0,0,0,0,1,0,0,0,0
4,13398,4.0,0.000217,5.0,0.000512,5.0,0.000523,3.0,0.000366,1.0,...,0,0,0,0,0,0,0,0,1,0


In [12]:
# X_vote_df.to_csv('../X_vote_df.csv', index=False)

In [13]:
X_vote_df.columns

Index(['index', 'QaA', 'QaE', 'QbA', 'QbE', 'QcA', 'QcE', 'QdA', 'QdE', 'QeA',
       'QeE', 'QfA', 'QfE', 'QgA', 'QgE', 'QhA', 'QhE', 'QiA', 'QiE', 'QjA',
       'QjE', 'QkA', 'QkE', 'QlA', 'QlE', 'QmA', 'QmE', 'QnA', 'QnE', 'QoA',
       'QoE', 'QpA', 'QpE', 'QqA', 'QqE', 'QrA', 'QrE', 'QsA', 'QsE', 'QtA',
       'QtE', 'age_group', 'education', 'familysize', 'married',
       'religion_Agnostic', 'religion_Atheist', 'religion_Buddhist',
       'religion_Christian_Catholic', 'religion_Christian_Mormon',
       'religion_Christian_Other', 'religion_Christian_Protestant',
       'religion_Hindu', 'religion_Jewish', 'religion_Muslim',
       'religion_Other', 'religion_Sikh'],
      dtype='object')

In [14]:
vote_df2 = transform_features(vote_df)

KeyError: "None of [Int64Index([1, 1, 1, 0, 1, 0, 1, 0, 0, 1,\n            ...\n            0, 0, 0, 0, 1, 1, 0, 1, 1, 1],\n           dtype='int64', length=36425)] are in the [columns]"

In [15]:
vote_df2.head()

,index,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,...,religion_Buddhist,religion_Christian_Catholic,religion_Christian_Mormon,religion_Christian_Other,religion_Christian_Protestant,religion_Hindu,religion_Jewish,religion_Muslim,religion_Other,religion_Sikh
0,43375,1.0,0.000124,5.0,0.000322,5.0,0.000383,1.0,0.000440,1.0,...,0,1,0,0,0,0,0,0,0,0
1,4486,1.0,0.000211,5.0,0.000473,5.0,0.002294,1.0,0.000690,1.0,...,0,0,0,0,0,1,0,0,0,0
2,10262,5.0,0.000242,1.0,0.000224,2.0,0.000687,1.0,0.001164,3.0,...,0,0,0,1,0,0,0,0,0,0
3,14088,4.0,0.000065,1.0,0.000911,1.0,0.002214,4.0,0.002965,1.0,...,0,0,0,0,0,1,0,0,0,0
4,13398,4.0,0.000217,5.0,0.000512,5.0,0.000523,3.0,0.000366,1.0,...,0,0,0,0,0,0,0,0,1,0


In [16]:
# vote_df2.to_csv('../vote_df2.csv', index=False)